In [1]:
!pip install --upgrade --quiet huggingface_hub

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
repo_id = "google/gemma-7b"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_new_tokens=512, temperature=0.5, 
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/charan/.cache/huggingface/token
Login successful


In [4]:
question = "Who won the FIFA World Cup in the year 2022? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [5]:

llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 

1. In the year 2022, the FIFA World Cup was held in Qatar. 

2. The final match was played between Argentina and France. 

3. Argentina won the match and the FIFA World Cup in the year 2022. 

Therefore, the answer to the question is Argentina. 

The FIFA World Cup is the biggest and most prestigious football tournament in the world. It is held every four years and features the best teams from around the globe. The 2022 edition of the tournament was held in Qatar and was won by Argentina, who defeated France in a thrilling final. The tournament was won by Argentina, who defeated France in a thrilling final. The final match was played at the Lusail Stadium in Doha, Qatar, and was watched by millions of fans around the world. 

The final match was a back-and-forth affair, with both teams trading goals and chances. In the end, it was Argentina who emerged victorious, with a score of 4-2 after extra time. The victory was especially sweet for Argentina, who had lost the final in 2014 to 

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = HuggingFaceBgeEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectordb.as_retriever()

In [7]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)

chain = create_stuff_documents_chain(llm, prompt)

docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

chain.invoke({"context": docs})

'\n\nJay loves yellow but not red\n\nJack loves orange but not blue\n\nJaden loves blue but not red\n\nJosh loves red but not blue\n\nJohnny loves green but not as much as he loves orange\n\nJoe loves orange but not red\n\nJames loves red but not green\n\nJames loves blue but not orange\n\nJosh loves blue but not red\n\nJamal loves yellow but not as much as he loves orange\n\nJack loves blue but not orange\n\nJaden loves green but not as much as he loves orange\n\nJosh loves green but not as much as he loves orange\n\nJohnny loves blue but not green\n\nJoe loves green but not as much as he loves orange\n\nJesse loves blue but not red\n\nJamal loves green but not as much as he loves orange\n\nJosh loves orange but not blue\n\nJohnny loves green but not as much as he loves orange\n\nJoe loves blue but not green\n\nJesse loves blue but not green\n\nJosh loves orange but not red\n\nJohnny loves green but not as much as he loves orange\n\nJoe loves orange but not green\n\nJesse loves orange

In [8]:
from langchain.chains import RetrievalQA
retrievalQA = RetrievalQA.from_llm(llm=llm, retriever=retriever)

In [11]:
retrievalQA.invoke("LLM Powered Autonomous Agents?")

{'query': 'LLM Powered Autonomous Agents?',
 'result': ' The answer is yes.\nUnhelpful Answer: The answer is no.\nUnhelpful Answer: The answer is maybe.\nUnhelpful Answer: The answer is probably.\nUnhelpful Answer: The answer is definitely.\nUnhelpful Answer: The answer is absolutely.\nUnhelpful Answer: The answer is certainly.\nUnhelpful Answer: The answer is undoubtedly.\nUnhelpful Answer: The answer is unquestionably.\nUnhelpful Answer: The answer is undoubtedly.\nUnhelpful Answer: The answer is undeniably.\nUnhelpful Answer: The answer is clearly.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nUnhelpful Answer: The answer is obviously.\nU